# VLM Failure Modes — Colab Execution

In [ ]:
# Setup & GPU Check
from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi

In [ ]:
# Load Project from Drive
%cd /content/drive/MyDrive
!ls
%cd vlm-failure-modes
!ls

In [ ]:
# Install Dependencies
!pip install --upgrade pip
!pip install -r requirements.txt

In [ ]:
# Install LLaVA (local editable)
!pip install -e LLaVA

In [ ]:
# Sanity Check (MANDATORY)
!python experiments/sanity_check.py

In [ ]:
# Entropy Analysis
!python experiments/entropy_analysis.py

In [ ]:
# (Optional) PGD Attack
!python attacks/pgd_visual.py

In [ ]:
# Verify Outputs
!ls results
!ls results/figures
!ls results/tables